In [278]:
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import itertools
import csv , codecs
import glob
from PIL import Image
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn
from sklearn import decomposition
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, log_loss



In [279]:
data_dir = "./EEG_data/"

## CSV 파일 생성 함수

In [280]:
def EEG_data_csv(data_dir):
    
    # Kinds of Power 입력
    file_list = os.listdir(data_dir)
    file_list.sort()
    print("Kinds of Power" ,file_list)
       
    global kinds_of_power
    kinds_of_power =str(input( "Kinds of Power : "))
    
    excel_dir = "./EEG_data" + '/' + kinds_of_power
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    # EEG_csv_data 디렉토리 생성
    csv_dir = "./EEG_csv_data"
    if not os.path.isdir(csv_dir):
        os.mkdir(csv_dir)
    
    Abs_dir ="./EEG_csv_data/Abs"
    Relative_dir = "./EEG_csv_data/Relative"
    
    if not os.path.isdir(Abs_dir):
        os.mkdir(Abs_dir)
    if not os.path.isdir(Relative_dir):
        os.mkdir(Relative_dir)
    
    book = Workbook()
    sheet = book.active
    
    # x축,y축 생성
    for i in range (1,21) :
        if i < 10 :
            sheet['A'+str(i+1)] = 'S0'+str(i)
        if i == 10 :
            sheet['A'+str(i+1)] = 'S'+str(10)
        if i > 10 :
            sheet['A'+str(i+1)] = 'S'+str(i)
            
    for j in range (0,12):
        for i in range(1,33):
            sheet.cell(row=1, column=160*j+i+1).value = 'ACh' + str(i)
            sheet.cell(row=1, column=160*j+i+33).value = 'BCh' + str(i)
            sheet.cell(row=1, column=160*j+i+65).value = 'DCh' + str(i)
            sheet.cell(row=1, column=160*j+i+97).value = 'GCh' + str(i)
            sheet.cell(row=1, column=160*j+i+129).value = 'TCh' + str(i)

    # 파일을 순차적으로 열어서 셀 영역을 복사
    # j: 엑셀 파일의 수, k: 각 엑셀 파일 당 시트의 수, n: task의 수, l: subject의 수, m: ch1-ch32

    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=l,column=160*k+m+32*j).value = source.cell(row=l,column=m).value

    # EEG_csv_data 디렉토리에 csv 파일로 저장
    if kinds_of_power == "Abs":
        with open(Abs_dir + str('/') +'Abs_Merged.csv', 'w', newline="") as f:
            c = csv.writer(f)
            for r in sheet.rows:
                c.writerow([cell.value for cell in r])
    elif kinds_of_power == "Relative":
        with open(Relative_dir + str('/') +'Relative_Merged.csv', 'w', newline="") as f:
            c = csv.writer(f)
            for r in sheet.rows:
                c.writerow([cell.value for cell in r])

## CSV 파일에서 Pandas dataframe으로 가져오는 함수

In [281]:
def data_setting(task, eye , respiratory):
    # 성재가 만든 파일 불러오는 경로 입력 및 데이터 프레임으로 불러올것 
    ##  Abs, Relative 설정은 여기서 변경하기기
    csv_name = "./EEG_csv_data/Abs/Abs_Merged.csv"
    data = pd.read_csv(csv_name) 
    data_column = data.columns
    
    
    # 1 back
    if task == 0 : 
        if respiratory == 0 :
            data = data[data_column[1:161]]
            # 1-back open eye mouse (alpha, beta, delta, gamma, theta   ch(32*5))
            
            
            
        elif respiratory == 1 :
            data = data[data_column[161:321]]
            # 1-back open eye nose (alpha, beta, delta, gamma, theta   ch(32*5))
        elif respiratory == 2 :
            data = data[data_column[321:481]]
            # 1-back open eye mouse o2 (alpha, beta, delta, gamma, theta   ch(32*5))
    # 2 back
    elif task == 1 : 
        if respiratory == 0 :
            data = data[data_column[481:641]]
            # 2-back open eye mouse (alpha, beta, delta, gamma, theta   ch(32*5))
        elif respiratory == 1 :
            data= data[data_column[641:801]]
            # 2-back open eye nose (alpha, beta, delta, gamma, theta   ch(32*5))
        elif respiratory == 2 :
            data = data[data_column[801:961]]
            # 2-back open eye mouse o2 (alpha, beta, delta, gamma, theta   ch(32*5))
    #resting
    elif task == 2 :
        ## eye ==0 close eye
        if eye == 0 :
            if respiratory == 0 :
                data = data[data_column[961:1121]]
                # resting open eye mouse (alpha, beta, delta, gamma, theta   ch(32*5))
            elif respiratory == 1 :
                data = data[data_column[1121:1281]]
                # resting open eye nose (alpha, beta, delta, gamma, theta   ch(32*5))
            elif respiratory == 2 :
                data = data[data_column[1281:1441]]
                # resting open eye mouse o2 (alpha, beta, delta, gamma, theta   ch(32*5))
        ## eye ==1 open eye
        if eye == 1 : 
            if respiratory == 0 :
                data = data[data_column[1441:1601]]
                # resting close eye mouse (alpha, beta, delta, gamma, theta   ch(32*5))
            elif respiratory == 1 :
                data = data[data_column[1601:1761]]
                # resting close eye nose (alpha, beta, delta, gamma, theta   ch(32*5))
            elif respiratory == 2 :
                data = data[data_column[1761:1921]]
                # resting close eye mouse o2 (alpha, beta, delta, gamma, theta   ch(32*5))

    
    
    
    
    return data
                
                
# column indexing task condition

### task list

In [282]:
wb = openpyxl.load_workbook(data_dir+ str('/') +'Relative'+str('/')+'R_Alpha.xlsx')
wb_sheet = wb.sheetnames
wb_sheet

['B1_M',
 'B1_N',
 'B1_O2',
 'B2_M',
 'B2_N',
 'B2_O2',
 'Rest_EC_M',
 'Rest_EC_N',
 'Rest_EC_O2',
 'Rest_EO_M',
 'Rest_EO_N',
 'Rest_EO_O2']

### Kinds of Power 입력

In [283]:
EEG_data_csv(data_dir)

Kinds of Power ['Abs', 'Relative']


Kinds of Power :  Relative



### task 0 1 2 : 1-back  2-back  rest

### eye 0  1    : close eye     open eye

### respiratory 0 1 2 : mouse nose o2 



---

### SVM dataset 만드는 함수

In [284]:
def dataset_csv():
    
    excel_dir = "./EEG_data" + '/' + "Abs"
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    subject = int(input("subject :"))
    

    book = Workbook()
    sheet = book.active
    sheet.cell(row=1, column=161).value = 'target'
 
    # x축 생성   
    for i in range(1,33):
        sheet.cell(row=1, column=i).value = 'ACh' + str(i)
        sheet.cell(row=1, column=i+32).value = 'BCh' + str(i)
        sheet.cell(row=1, column=i+64).value = 'DCh' + str(i)
        sheet.cell(row=1, column=i+96).value = 'GCh' + str(i)
        sheet.cell(row=1, column=i+128).value = 'TCh' + str(i)
        
    # 파일을 순차적으로 열어서 셀 영역을 복사
    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=20*k+l,column=32*j+m-1).value = source.cell(row=l,column=m).value
        

    with open('./dataset_eeg.csv', 'w', newline="") as f:
        c = csv.writer(f)
        for r in sheet.rows:
            c.writerow([cell.value for cell in r])
            
    df = pd.read_csv('./dataset_eeg.csv')
    df[df =='                      NaN'] =np.nan
    df = df.fillna(0.0).astype('float64')
    pd.options.mode.chained_assignment = None

    df["target"][0*subject:1*subject] = "010"
    df["target"][1*subject:2*subject] = "011"
    df["target"][2*subject:3*subject] = "012"

    df["target"][3*subject:4*subject] = "110"
    df["target"][4*subject:5*subject] = "111"
    df["target"][5*subject:6*subject] = "112"

    df["target"][6*subject:7*subject] = "200"
    df["target"][7*subject:8*subject] = "201"
    df["target"][8*subject:9*subject] = "202"

    df["target"][9*subject:10*subject] = "210"
    df["target"][10*subject:11*subject] = "211"
    df["target"][11*subject:12*subject] = "212"
    df.astype('float64')
    df.to_csv('./dataset_eeg.csv')
    
    return df
            


In [285]:
eeg_data = dataset_csv()

subject : 20


C:\Users\hyg44\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [286]:
eeg_data

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32,target
0,34.791231,43.380524,35.326932,33.483708,56.872380,80.370204,214.133509,336.869775,381.856303,273.700635,...,13.427554,8.834771,14.327464,14.443761,7.069934,2.461342,1.918468,3.426660,2.290185,010
1,21.332838,24.823823,7.732816,9.217822,9.089609,8.036041,15.609924,23.939677,45.026376,32.271230,...,13.955683,23.515145,33.013509,51.655861,26.336138,4.967240,4.443262,4.745022,4.208323,010
2,36.186846,30.996535,10.170486,9.205632,7.786313,11.105612,15.048448,23.943661,62.093379,51.073027,...,5.010105,15.810370,20.048230,32.001265,10.859633,1.375167,1.457062,1.650200,1.416968,010
3,165.706432,160.501820,53.815336,62.235116,55.541734,71.253082,91.474654,135.868921,134.070627,136.141713,...,16.361836,68.210183,54.308069,148.442087,63.703027,7.818248,6.864512,4.247089,4.067750,010
4,22.634142,32.687601,12.911995,12.740504,10.412052,13.157750,18.063993,45.134604,28.956921,28.032936,...,6.803263,10.276619,11.929224,19.722379,8.552856,2.337217,1.963548,1.735394,1.721120,010
5,9.904728,12.106190,5.481018,5.883165,6.943520,4.971094,11.073126,21.883341,28.940616,17.855794,...,5.201979,8.331477,9.298878,14.191144,6.611382,2.270006,1.609459,2.243871,1.464934,010
6,18.326832,18.923015,11.541991,11.720848,21.818624,16.801940,44.238363,45.513382,86.959537,96.298758,...,10.462003,6.948912,16.636356,19.600377,11.220615,4.065819,3.602533,4.167830,3.110982,010
7,24.845787,25.098427,9.968054,10.518397,11.365719,14.987010,34.950889,85.022078,65.631135,67.088813,...,13.238787,30.234171,23.676020,44.214361,45.812588,7.498409,4.584952,4.489311,4.936536,010
8,40.920388,38.523930,20.494293,16.728225,22.788190,17.159516,46.666191,66.393587,77.053292,70.261983,...,6.364726,11.050834,11.586133,21.515216,8.912505,2.377013,2.058713,2.536856,1.795944,010
9,38.508282,42.220113,8.909832,10.129484,11.125612,11.333431,59.035223,71.950390,127.573202,117.084965,...,4.425120,8.617088,9.871563,17.264892,6.362957,1.312544,1.445537,1.684411,1.398925,010


In [287]:
X = eeg_data.iloc[:,0:-1]
X

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
0,34.791231,43.380524,35.326932,33.483708,56.872380,80.370204,214.133509,336.869775,381.856303,273.700635,...,13.007852,13.427554,8.834771,14.327464,14.443761,7.069934,2.461342,1.918468,3.426660,2.290185
1,21.332838,24.823823,7.732816,9.217822,9.089609,8.036041,15.609924,23.939677,45.026376,32.271230,...,14.054909,13.955683,23.515145,33.013509,51.655861,26.336138,4.967240,4.443262,4.745022,4.208323
2,36.186846,30.996535,10.170486,9.205632,7.786313,11.105612,15.048448,23.943661,62.093379,51.073027,...,5.304896,5.010105,15.810370,20.048230,32.001265,10.859633,1.375167,1.457062,1.650200,1.416968
3,165.706432,160.501820,53.815336,62.235116,55.541734,71.253082,91.474654,135.868921,134.070627,136.141713,...,16.519616,16.361836,68.210183,54.308069,148.442087,63.703027,7.818248,6.864512,4.247089,4.067750
4,22.634142,32.687601,12.911995,12.740504,10.412052,13.157750,18.063993,45.134604,28.956921,28.032936,...,6.691182,6.803263,10.276619,11.929224,19.722379,8.552856,2.337217,1.963548,1.735394,1.721120
5,9.904728,12.106190,5.481018,5.883165,6.943520,4.971094,11.073126,21.883341,28.940616,17.855794,...,7.218875,5.201979,8.331477,9.298878,14.191144,6.611382,2.270006,1.609459,2.243871,1.464934
6,18.326832,18.923015,11.541991,11.720848,21.818624,16.801940,44.238363,45.513382,86.959537,96.298758,...,14.291189,10.462003,6.948912,16.636356,19.600377,11.220615,4.065819,3.602533,4.167830,3.110982
7,24.845787,25.098427,9.968054,10.518397,11.365719,14.987010,34.950889,85.022078,65.631135,67.088813,...,14.024846,13.238787,30.234171,23.676020,44.214361,45.812588,7.498409,4.584952,4.489311,4.936536
8,40.920388,38.523930,20.494293,16.728225,22.788190,17.159516,46.666191,66.393587,77.053292,70.261983,...,8.599619,6.364726,11.050834,11.586133,21.515216,8.912505,2.377013,2.058713,2.536856,1.795944
9,38.508282,42.220113,8.909832,10.129484,11.125612,11.333431,59.035223,71.950390,127.573202,117.084965,...,4.805566,4.425120,8.617088,9.871563,17.264892,6.362957,1.312544,1.445537,1.684411,1.398925


In [288]:
X.shape

(240, 160)

In [289]:
X.columns

Index(['ACh1', 'ACh2', 'ACh3', 'ACh4', 'ACh5', 'ACh6', 'ACh7', 'ACh8', 'ACh9',
       'ACh10',
       ...
       'TCh23', 'TCh24', 'TCh25', 'TCh26', 'TCh27', 'TCh28', 'TCh29', 'TCh30',
       'TCh31', 'TCh32'],
      dtype='object', length=160)

In [290]:
X.dtypes[X.dtypes == 'float64']

ACh1     float64
ACh2     float64
ACh3     float64
ACh4     float64
ACh5     float64
ACh6     float64
ACh7     float64
ACh8     float64
ACh9     float64
ACh10    float64
ACh11    float64
ACh12    float64
ACh13    float64
ACh14    float64
ACh15    float64
ACh16    float64
ACh17    float64
ACh18    float64
ACh19    float64
ACh20    float64
ACh21    float64
ACh22    float64
ACh23    float64
ACh24    float64
ACh25    float64
ACh26    float64
ACh27    float64
ACh28    float64
ACh29    float64
ACh30    float64
          ...   
TCh3     float64
TCh4     float64
TCh5     float64
TCh6     float64
TCh7     float64
TCh8     float64
TCh9     float64
TCh10    float64
TCh11    float64
TCh12    float64
TCh13    float64
TCh14    float64
TCh15    float64
TCh16    float64
TCh17    float64
TCh18    float64
TCh19    float64
TCh20    float64
TCh21    float64
TCh22    float64
TCh23    float64
TCh24    float64
TCh25    float64
TCh26    float64
TCh27    float64
TCh28    float64
TCh29    float64
TCh30    float

In [291]:
X.describe()

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
count,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,...,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000
mean,42.561788,47.472589,19.689349,18.449647,22.535127,22.090712,49.556503,63.022651,79.552800,73.504101,...,14.880760,12.990100,23.722410,26.522233,51.223462,21.758711,4.576625,4.281536,4.530858,3.473750
std,69.564224,80.101041,23.503146,14.160734,18.199340,19.505364,46.940654,69.442135,76.457327,61.274676,...,16.156164,12.409036,25.991041,30.788968,78.212551,25.992692,5.260781,5.768968,5.123915,3.227539
min,0.000000,3.552270,4.158723,4.236558,4.157920,4.311342,4.788523,6.172562,6.490573,5.710638,...,3.574281,3.222559,2.681652,3.156142,2.477897,2.676950,0.986446,0.850590,1.226467,0.795973
25%,11.874460,12.775302,8.639672,8.617056,8.878504,9.592497,19.963195,24.296084,32.417793,28.424702,...,7.370792,6.459315,8.822791,9.891404,12.985007,6.842809,2.013142,1.756304,2.176812,1.685248
50%,20.016081,23.679530,14.136242,13.710465,16.749443,14.814985,34.950268,44.455078,59.082971,54.458988,...,9.939426,9.636837,14.659191,17.323115,25.497461,12.477759,2.828906,2.523049,3.003323,2.416691
75%,45.132109,44.902270,21.367175,21.812000,29.404307,25.639561,59.072721,76.661113,102.069204,105.126630,...,14.698463,14.059344,28.810445,28.465749,50.111478,24.307156,5.011866,4.582901,4.347526,4.064372
max,569.261488,645.963819,288.897707,62.235116,110.015938,138.112567,353.473941,687.843902,704.253173,477.610148,...,117.605093,91.114884,175.855475,189.560378,582.296101,160.771828,42.455278,50.339229,40.610675,23.860095


In [292]:
X.head()

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
0,34.791231,43.380524,35.326932,33.483708,56.872380,80.370204,214.133509,336.869775,381.856303,273.700635,...,13.007852,13.427554,8.834771,14.327464,14.443761,7.069934,2.461342,1.918468,3.426660,2.290185
1,21.332838,24.823823,7.732816,9.217822,9.089609,8.036041,15.609924,23.939677,45.026376,32.271230,...,14.054909,13.955683,23.515145,33.013509,51.655861,26.336138,4.967240,4.443262,4.745022,4.208323
2,36.186846,30.996535,10.170486,9.205632,7.786313,11.105612,15.048448,23.943661,62.093379,51.073027,...,5.304896,5.010105,15.810370,20.048230,32.001265,10.859633,1.375167,1.457062,1.650200,1.416968
3,165.706432,160.501820,53.815336,62.235116,55.541734,71.253082,91.474654,135.868921,134.070627,136.141713,...,16.519616,16.361836,68.210183,54.308069,148.442087,63.703027,7.818248,6.864512,4.247089,4.067750
4,22.634142,32.687601,12.911995,12.740504,10.412052,13.157750,18.063993,45.134604,28.956921,28.032936,...,6.691182,6.803263,10.276619,11.929224,19.722379,8.552856,2.337217,1.963548,1.735394,1.721120


In [293]:
X.tail()

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
235,18.538067,26.082164,6.797062,7.368925,6.505240,6.335676,10.269693,13.413973,30.098354,25.573889,...,7.300164,8.440227,11.895827,19.511463,26.077071,13.111850,2.429150,2.009444,1.925261,1.643929
236,13.858100,14.924126,8.753701,9.650271,10.556247,10.906756,20.970709,18.413634,33.556794,26.609574,...,8.106977,7.205655,9.257661,9.660021,11.286730,4.819451,1.852064,1.576015,2.527010,2.054019
237,57.411774,69.868794,32.129234,35.201466,41.671562,47.887935,54.403943,59.316903,60.315386,60.738227,...,35.736438,31.428706,38.400018,48.301033,77.324785,36.430481,8.534439,8.120699,14.052555,10.501860
238,19.762112,18.894355,12.118001,8.936352,18.889075,12.277530,23.898682,20.512957,32.630829,29.396120,...,14.931026,11.586026,14.384869,10.305868,15.755834,10.283204,2.831115,3.142641,3.736056,2.544786
239,5.547198,5.640918,7.726326,5.602781,14.813791,10.307345,20.786167,13.897660,28.975541,23.475854,...,7.313346,7.767707,4.738413,3.156142,2.477897,3.633768,2.391836,2.154864,2.906150,2.210984


In [294]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Columns: 160 entries, ACh1 to TCh32
dtypes: float64(160)
memory usage: 300.1 KB


In [295]:
Y = eeg_data['target']
Y

0      010
1      010
2      010
3      010
4      010
5      010
6      010
7      010
8      010
9      010
10     010
11     010
12     010
13     010
14     010
15     010
16     010
17     010
18     010
19     010
20     011
21     011
22     011
23     011
24     011
25     011
26     011
27     011
28     011
29     011
      ... 
210    211
211    211
212    211
213    211
214    211
215    211
216    211
217    211
218    211
219    211
220    212
221    212
222    212
223    212
224    212
225    212
226    212
227    212
228    212
229    212
230    212
231    212
232    212
233    212
234    212
235    212
236    212
237    212
238    212
239    212
Name: target, Length: 240, dtype: object

In [296]:
Y.shape

(240,)

In [297]:
?SVC

Init signature:
SVC(
    C=1.0,
    kernel='rbf',
    degree=3,
    gamma='auto_deprecated',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=0.001,
    cache_size=200,
    class_weight=None,
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovr',
    random_state=None,
)
Docstring:     
C-Support Vector Classification.

The implementation is based on libsvm. The fit time scales at least
quadratically with the number of samples and may be impractical
beyond tens of thousands of samples. For large datasets
consider using :class:`sklearn.linear_model.LinearSVC` or
:class:`sklearn.linear_model.SGDClassifier` instead, possibly after a
:class:`sklearn.kernel_approximation.Nystroem` transformer.

The multiclass support is handled according to a one-vs-one scheme.

For details on the precise mathematical formulation of the provided
kernel functions and how `gamma`, `coef0` and `degree` affect each
other, see the corresponding section in the narrative documentat

In [298]:
X= np.array(X)
Y= np.array(Y)

In [299]:
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)

all_scores = []

model = Pipeline((("scaler", StandardScaler()),("svm_clf", SVC(kernel='rbf',gamma='scale', decision_function_shape='ovo'))))


for train_index, test_index in skf.split(X,Y):
    print("처리중인 폴드 #")
    print("TEST : ", test_index, test_index.shape,"\n")
    model.fit(X[train_index],Y[train_index])
    print("-------------\n")
    print(model.predict(X[test_index]),"\n",Y[test_index],"\n")
    val_mae = model.score(X[test_index],Y[test_index])
    all_scores.append(val_mae)
    print("-------------\n")
    
print(all_scores)



처리중인 폴드 #
TEST :  [  2  13  30  36  51  58  69  70  80  97 108 119 134 139 142 145 167 169
 186 199 213 216 220 225] (24,) 

-------------

['110' '202' '110' '210' '011' '110' '112' '112' '012' '112' '110' '012'
 '202' '012' '202' '110' '202' '201' '011' '212' '012' '110' '202' '111'] 
 ['010' '010' '011' '011' '012' '012' '110' '110' '111' '111' '112' '112'
 '200' '200' '201' '201' '202' '202' '210' '210' '211' '211' '212' '212'] 

-------------

처리중인 폴드 #
TEST :  [  4  10  22  38  43  56  72  73  91  93 102 104 124 136 140 148 168 173
 181 194 212 215 236 239] (24,) 

-------------

['112' '110' '211' '202' '211' '201' '110' '202' '011' '200' '110' '110'
 '110' '110' '202' '110' '110' '201' '212' '211' '112' '110' '110' '010'] 
 ['010' '010' '011' '011' '012' '012' '110' '110' '111' '111' '112' '112'
 '200' '200' '201' '201' '202' '202' '210' '210' '211' '211' '212' '212'] 

-------------

처리중인 폴드 #
TEST :  [  3   8  26  29  48  50  67  76  81  87 103 107 122 125 144 154 162 175
 18

In [300]:
print(np.mean(all_scores)*100,"%")

4.583333333333334 %


---

# Min-Max normalization을 진행

In [301]:
X = eeg_data.iloc[:,0:-1]
minimum = X.min(axis=0)
X -= minimum
std = X.max(axis=0) - X.min(axis =0)
X /= std
X1 = pd.DataFrame(X)
X1.describe()

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
count,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,...,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000
mean,0.074767,0.068368,0.054543,0.245059,0.173602,0.132879,0.128391,0.083398,0.104709,0.143661,...,0.099153,0.111131,0.121501,0.125352,0.084070,0.120698,0.086575,0.069328,0.083901,0.116101
std,0.122201,0.124688,0.082543,0.244157,0.171922,0.145779,0.134622,0.101870,0.109575,0.129847,...,0.141682,0.141185,0.150086,0.165173,0.134892,0.164412,0.126861,0.116572,0.130101,0.139938
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.020859,0.014357,0.015737,0.075528,0.044594,0.039470,0.043520,0.026587,0.037158,0.048133,...,0.033294,0.036826,0.035462,0.036133,0.018121,0.026350,0.024758,0.018301,0.024130,0.038557
50%,0.035161,0.031331,0.035041,0.163347,0.118947,0.078502,0.086501,0.056160,0.075373,0.103302,...,0.055820,0.072979,0.069165,0.076001,0.039701,0.061993,0.044430,0.033795,0.045116,0.070270
75%,0.079282,0.064367,0.060436,0.303032,0.238493,0.159402,0.155682,0.103405,0.136979,0.210672,...,0.097554,0.123296,0.150882,0.135778,0.082153,0.136818,0.097071,0.075418,0.079246,0.141709
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [302]:
X1[X1.columns].isnull().values.any()

True

In [303]:
X1 = X1.fillna(0.0)

In [304]:
X1.shape

(240, 160)

In [305]:
Y = eeg_data['target']
Y

0      010
1      010
2      010
3      010
4      010
5      010
6      010
7      010
8      010
9      010
10     010
11     010
12     010
13     010
14     010
15     010
16     010
17     010
18     010
19     010
20     011
21     011
22     011
23     011
24     011
25     011
26     011
27     011
28     011
29     011
      ... 
210    211
211    211
212    211
213    211
214    211
215    211
216    211
217    211
218    211
219    211
220    212
221    212
222    212
223    212
224    212
225    212
226    212
227    212
228    212
229    212
230    212
231    212
232    212
233    212
234    212
235    212
236    212
237    212
238    212
239    212
Name: target, Length: 240, dtype: object

In [306]:
Y.shape

(240,)

In [307]:
X= np.array(X1)
Y= np.array(Y)

In [308]:
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)

all_scores = []

model = SVC(kernel='rbf',gamma='auto', decision_function_shape='ovo')


for train_index, test_index in skf.split(X,Y):
    model.fit(X[train_index],Y[train_index])
    val_mae = model.score(X[test_index],Y[test_index])
    all_scores.append(val_mae)
    
print(all_scores)



[0.08333333333333333, 0.041666666666666664, 0.08333333333333333, 0.041666666666666664, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.0, 0.0, 0.0]


In [309]:
print(np.mean(all_scores)*100,"%")

5.0 %


---

# 1. code 

---
### task 0 1 2 : 1-back  2-back  rest

### eye 0  1    : close eye     open eye

### respiratory 0 1 2 : mouse nose o2 
---






## 010 'B1_M',


## 011 'B1_N',


## 012 'B1_O2'



3 종류 task 구분

In [310]:
X = eeg_data.iloc[0:60,0:-1]
minimum = X.min(axis=0)
X -= minimum
std = X.max(axis=0) - X.min(axis =0)
X /= std
X1 = pd.DataFrame(X)
X1 = X1.fillna(0.0)
X1.describe()

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.088230,0.086804,0.206918,0.260847,0.233877,0.232282,0.218330,0.174904,0.199093,0.272711,...,0.105716,0.140273,0.192156,0.193259,0.127831,0.158130,0.122047,0.089794,0.115684,0.154175
std,0.154255,0.153859,0.214467,0.253887,0.218174,0.239685,0.209003,0.191041,0.196249,0.217971,...,0.162226,0.168906,0.212784,0.229043,0.191444,0.203687,0.172976,0.168305,0.171730,0.173788
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.024235,0.024531,0.073759,0.091716,0.083977,0.099920,0.104529,0.064744,0.089115,0.134766,...,0.037165,0.050351,0.059000,0.058296,0.031767,0.038923,0.043125,0.018909,0.035591,0.058643
50%,0.052846,0.043586,0.121995,0.175423,0.163910,0.140792,0.152777,0.119366,0.149557,0.213295,...,0.054920,0.095312,0.122280,0.130740,0.076599,0.087337,0.064067,0.030022,0.068972,0.093286
75%,0.074629,0.077950,0.240104,0.313721,0.291854,0.236643,0.243937,0.210066,0.242493,0.402220,...,0.098475,0.161682,0.245861,0.203936,0.116796,0.168092,0.152445,0.094439,0.114848,0.203150
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [311]:
Y = eeg_data['target'][0:60]
Y

0     010
1     010
2     010
3     010
4     010
5     010
6     010
7     010
8     010
9     010
10    010
11    010
12    010
13    010
14    010
15    010
16    010
17    010
18    010
19    010
20    011
21    011
22    011
23    011
24    011
25    011
26    011
27    011
28    011
29    011
30    011
31    011
32    011
33    011
34    011
35    011
36    011
37    011
38    011
39    011
40    012
41    012
42    012
43    012
44    012
45    012
46    012
47    012
48    012
49    012
50    012
51    012
52    012
53    012
54    012
55    012
56    012
57    012
58    012
59    012
Name: target, dtype: object

In [312]:
X= np.array(X1)
Y= np.array(Y)

In [313]:
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)

all_scores = []

model = SVC(kernel='rbf',gamma='auto', decision_function_shape='ovo')


for train_index, test_index in skf.split(X,Y):
    model.fit(X[train_index],Y[train_index])
    val_mae = model.score(X[test_index],Y[test_index])
    all_scores.append(val_mae)
    
print(all_scores)



[0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.3333333333333333, 0.3333333333333333]


In [314]:
print(np.mean(all_scores)*100,"%")

25.0 %


---

# 2. code 

---
### task 0 1 2 : 1-back  2-back  rest

### eye 0  1    : close eye     open eye

### respiratory 0 1 2 : mouse nose o2 
---


## 110 'B2_M',


## 111 'B2_N',


## 112 'B2_O2'



3 종류 task 구분

In [315]:
X = eeg_data.iloc[60:120,0:-1]
minimum = X.min(axis=0)
X -= minimum
std = X.max(axis=0) - X.min(axis =0)
X /= std
X1 = pd.DataFrame(X)
X1 = X1.fillna(0.0)
X1.describe()

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.090465,0.099160,0.189317,0.243419,0.214977,0.177556,0.151796,0.133444,0.151827,0.192030,...,0.186602,0.189710,0.137090,0.147410,0.110398,0.149691,0.154762,0.169477,0.214498,0.241542
std,0.174508,0.187012,0.204138,0.259645,0.233344,0.210135,0.176073,0.171750,0.173003,0.188974,...,0.164320,0.166747,0.184146,0.202401,0.192224,0.202944,0.169387,0.171936,0.181438,0.190608
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.013825,0.016690,0.045763,0.045673,0.045169,0.045771,0.047860,0.042771,0.057094,0.070359,...,0.098133,0.083000,0.041128,0.036695,0.019504,0.028019,0.057027,0.058864,0.106279,0.103508
50%,0.035165,0.037860,0.131597,0.170999,0.153517,0.114525,0.103659,0.088382,0.109176,0.132597,...,0.155687,0.161151,0.073751,0.091627,0.049009,0.068818,0.088358,0.097527,0.150402,0.178617
75%,0.076701,0.069990,0.253712,0.311268,0.282789,0.176228,0.178783,0.150174,0.176206,0.248004,...,0.224327,0.229963,0.145005,0.152499,0.085679,0.160835,0.200493,0.245939,0.279107,0.368562
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [316]:
Y = eeg_data['target'][60:120]
Y

60     110
61     110
62     110
63     110
64     110
65     110
66     110
67     110
68     110
69     110
70     110
71     110
72     110
73     110
74     110
75     110
76     110
77     110
78     110
79     110
80     111
81     111
82     111
83     111
84     111
85     111
86     111
87     111
88     111
89     111
90     111
91     111
92     111
93     111
94     111
95     111
96     111
97     111
98     111
99     111
100    112
101    112
102    112
103    112
104    112
105    112
106    112
107    112
108    112
109    112
110    112
111    112
112    112
113    112
114    112
115    112
116    112
117    112
118    112
119    112
Name: target, dtype: object

In [317]:
X= np.array(X1)
Y= np.array(Y)

In [318]:
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)

all_scores = []

model = SVC(kernel='rbf',gamma='auto', decision_function_shape='ovo')


for train_index, test_index in skf.split(X,Y):
    model.fit(X[train_index],Y[train_index])
    val_mae = model.score(X[test_index],Y[test_index])
    all_scores.append(val_mae)
    
print(all_scores)



[0.16666666666666666, 0.3333333333333333, 0.16666666666666666, 0.0, 0.0, 0.16666666666666666, 0.16666666666666666, 0.0, 0.16666666666666666, 0.0]


In [319]:
print(np.mean(all_scores)*100,"%")

11.666666666666666 %


# 3. code 

---
### task 0 1 2 : 1-back  2-back  rest

### eye 0  1    : close eye     open eye

### respiratory 0 1 2 : mouse nose o2 
---


## 200  'Rest_EC_M'


## 201 'Rest_EC_N',


## 202  'Rest_EC_O2'



3 종류 task 구분

In [320]:
X = eeg_data.iloc[120:180,0:-1]
minimum = X.min(axis=0)
X -= minimum
std = X.max(axis=0) - X.min(axis =0)
X /= std
X1 = pd.DataFrame(X)
X1 = X1.fillna(0.0)
X1.describe()

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.305247,0.245005,0.071314,0.254290,0.226939,0.171111,0.168480,0.106127,0.126604,0.173686,...,0.176957,0.183906,0.158899,0.144942,0.139418,0.143378,0.134011,0.129075,0.134929,0.187931
std,0.220181,0.245715,0.141443,0.257832,0.227922,0.178179,0.186053,0.139588,0.148085,0.169616,...,0.234769,0.220143,0.205172,0.187421,0.196240,0.196662,0.195266,0.188905,0.203320,0.226477
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.154827,0.094307,0.013835,0.069675,0.065781,0.035336,0.048347,0.027470,0.027515,0.037923,...,0.037748,0.054500,0.043738,0.046018,0.044597,0.030413,0.025602,0.030133,0.017716,0.059567
50%,0.248819,0.175035,0.039874,0.172058,0.159272,0.121981,0.110989,0.078220,0.084043,0.126324,...,0.094975,0.117704,0.076430,0.077987,0.068950,0.060540,0.061254,0.060100,0.064507,0.094098
75%,0.353907,0.294774,0.059902,0.312779,0.281550,0.220958,0.196105,0.139243,0.184290,0.309777,...,0.163929,0.206597,0.187166,0.164944,0.126099,0.166982,0.154304,0.138341,0.120226,0.225791
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [321]:
Y = eeg_data['target'][120:180]
Y

120    200
121    200
122    200
123    200
124    200
125    200
126    200
127    200
128    200
129    200
130    200
131    200
132    200
133    200
134    200
135    200
136    200
137    200
138    200
139    200
140    201
141    201
142    201
143    201
144    201
145    201
146    201
147    201
148    201
149    201
150    201
151    201
152    201
153    201
154    201
155    201
156    201
157    201
158    201
159    201
160    202
161    202
162    202
163    202
164    202
165    202
166    202
167    202
168    202
169    202
170    202
171    202
172    202
173    202
174    202
175    202
176    202
177    202
178    202
179    202
Name: target, dtype: object

In [322]:
X= np.array(X1)
Y= np.array(Y)

In [323]:
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)

all_scores = []

model = SVC(kernel='rbf',gamma='auto', decision_function_shape='ovo')


for train_index, test_index in skf.split(X,Y):
    model.fit(X[train_index],Y[train_index])
    val_mae = model.score(X[test_index],Y[test_index])
    all_scores.append(val_mae)
    
print(all_scores)



[0.0, 0.16666666666666666, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666, 0.3333333333333333, 0.16666666666666666, 0.3333333333333333, 0.3333333333333333]


In [324]:
print(np.mean(all_scores)*100,"%")

25.0 %


# 4. code 

---
### task 0 1 2 : 1-back  2-back  rest

### eye 0  1    : close eye     open eye

### respiratory 0 1 2 : mouse nose o2 
---


## 210  'Rest_EO_M'


## 211 'Rest_EO_N',


## 212  'Rest_EO_O2'



3 종류 task 구분

In [325]:
X = eeg_data.iloc[180:240,0:-1]
minimum = X.min(axis=0)
X -= minimum
std = X.max(axis=0) - X.min(axis =0)
X /= std
X1 = pd.DataFrame(X)
X1 = X1.fillna(0.0)
X1.describe()

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.243825,0.181605,0.246227,0.252233,0.313705,0.207720,0.221247,0.150882,0.192828,0.235532,...,0.092329,0.099023,0.137304,0.135342,0.169438,0.115275,0.080711,0.065605,0.074511,0.104160
std,0.258145,0.205416,0.253917,0.263080,0.289751,0.238592,0.228904,0.203906,0.213097,0.242474,...,0.152899,0.154895,0.184706,0.191650,0.222388,0.170821,0.151209,0.147815,0.157104,0.154413
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.060207,0.040177,0.075640,0.072693,0.078965,0.065153,0.064632,0.038372,0.068889,0.075804,...,0.032241,0.033567,0.031781,0.036913,0.033880,0.024057,0.020072,0.013166,0.009847,0.030601
50%,0.126292,0.104425,0.142485,0.144221,0.197275,0.105437,0.115553,0.078937,0.108958,0.132665,...,0.052038,0.059762,0.068568,0.076760,0.081438,0.056486,0.044848,0.029063,0.030117,0.061930
75%,0.335277,0.233207,0.308199,0.317477,0.489191,0.250995,0.323616,0.162746,0.225539,0.305627,...,0.087412,0.103345,0.180448,0.153790,0.211710,0.148918,0.080342,0.056469,0.056796,0.114432
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [326]:
Y = eeg_data['target'][180:240]
Y

180    210
181    210
182    210
183    210
184    210
185    210
186    210
187    210
188    210
189    210
190    210
191    210
192    210
193    210
194    210
195    210
196    210
197    210
198    210
199    210
200    211
201    211
202    211
203    211
204    211
205    211
206    211
207    211
208    211
209    211
210    211
211    211
212    211
213    211
214    211
215    211
216    211
217    211
218    211
219    211
220    212
221    212
222    212
223    212
224    212
225    212
226    212
227    212
228    212
229    212
230    212
231    212
232    212
233    212
234    212
235    212
236    212
237    212
238    212
239    212
Name: target, dtype: object

In [327]:
X= np.array(X1)
Y= np.array(Y)

In [328]:
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)

all_scores = []

model = SVC(kernel='rbf',gamma='auto', decision_function_shape='ovo')


for train_index, test_index in skf.split(X,Y):
    model.fit(X[train_index],Y[train_index])
    val_mae = model.score(X[test_index],Y[test_index])
    all_scores.append(val_mae)
    
print(all_scores)



[0.0, 0.16666666666666666, 0.16666666666666666, 0.0, 0.16666666666666666, 0.3333333333333333, 0.0, 0.16666666666666666, 0.0, 0.16666666666666666]


In [329]:
print(np.mean(all_scores)*100,"%")

11.666666666666666 %


# 12 task

In [330]:
X = eeg_data.iloc[:,0:-1]
X=np.array(X)

In [331]:
Y = eeg_data['target']
Y=np.array(Y)

In [332]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}
                    ,{'kernel': ['sigmoid'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}
                    ,{'kernel': ['linear'], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}
                   ]

scores = ['precision']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=10, scoring='%s_macro' % score)
    clf.fit(X, Y)
    print("Best parameters set found on development set:")
    print(clf.best_params_)
    print("Grid scores on development set:")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

# Tuning hyper-parameters for precision
Best parameters set found on development set:
{'C': 0.1, 'kernel': 'linear'}
Grid scores on development set:
0.069 (+/-0.104) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'}
0.031 (+/-0.050) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
0.035 (+/-0.054) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'rbf'}
0.039 (+/-0.074) for {'C': 0.001, 'gamma': 1e-05, 'kernel': 'rbf'}
0.069 (+/-0.104) for {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
0.027 (+/-0.049) for {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
0.035 (+/-0.054) for {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.039 (+/-0.074) for {'C': 0.1, 'gamma': 1e-05, 'kernel': 'rbf'}
0.069 (+/-0.104) for {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
0.027 (+/-0.049) for {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
0.035 (+/-0.054) for {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.039 (+/-0.074) for {'C': 0.1, 'gamma': 1e-05, 'kernel': 'rbf'}
0.016 (+/-0.034) for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [333]:
clf.best_params_

{'C': 0.1, 'kernel': 'linear'}

# 3 task

In [334]:
X = eeg_data.iloc[180:240,0:-1]
minimum = X.min(axis=0)
X -= minimum
std = X.max(axis=0) - X.min(axis =0)
X /= std
X1 = pd.DataFrame(X)
X1 = X1.fillna(0.0)
X1.describe()

,ACh1,ACh2,ACh3,ACh4,ACh5,ACh6,ACh7,ACh8,ACh9,ACh10,...,TCh23,TCh24,TCh25,TCh26,TCh27,TCh28,TCh29,TCh30,TCh31,TCh32
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.243825,0.181605,0.246227,0.252233,0.313705,0.207720,0.221247,0.150882,0.192828,0.235532,...,0.092329,0.099023,0.137304,0.135342,0.169438,0.115275,0.080711,0.065605,0.074511,0.104160
std,0.258145,0.205416,0.253917,0.263080,0.289751,0.238592,0.228904,0.203906,0.213097,0.242474,...,0.152899,0.154895,0.184706,0.191650,0.222388,0.170821,0.151209,0.147815,0.157104,0.154413
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.060207,0.040177,0.075640,0.072693,0.078965,0.065153,0.064632,0.038372,0.068889,0.075804,...,0.032241,0.033567,0.031781,0.036913,0.033880,0.024057,0.020072,0.013166,0.009847,0.030601
50%,0.126292,0.104425,0.142485,0.144221,0.197275,0.105437,0.115553,0.078937,0.108958,0.132665,...,0.052038,0.059762,0.068568,0.076760,0.081438,0.056486,0.044848,0.029063,0.030117,0.061930
75%,0.335277,0.233207,0.308199,0.317477,0.489191,0.250995,0.323616,0.162746,0.225539,0.305627,...,0.087412,0.103345,0.180448,0.153790,0.211710,0.148918,0.080342,0.056469,0.056796,0.114432
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [335]:
Y = eeg_data['target'][180:240]
Y

180    210
181    210
182    210
183    210
184    210
185    210
186    210
187    210
188    210
189    210
190    210
191    210
192    210
193    210
194    210
195    210
196    210
197    210
198    210
199    210
200    211
201    211
202    211
203    211
204    211
205    211
206    211
207    211
208    211
209    211
210    211
211    211
212    211
213    211
214    211
215    211
216    211
217    211
218    211
219    211
220    212
221    212
222    212
223    212
224    212
225    212
226    212
227    212
228    212
229    212
230    212
231    212
232    212
233    212
234    212
235    212
236    212
237    212
238    212
239    212
Name: target, dtype: object

In [336]:
X= np.array(X1)
Y= np.array(Y)

In [337]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}
                    #,{'kernel': ['sigmoid'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}
                    #,{'kernel': ['linear'], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}
                   ]

scores = ['precision']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    clf = GridSearchCV(SVC(), tuned_parameters, cv=10, scoring='%s_macro' % score)
    clf.fit(X, Y)
    print("Best parameters set found on development set:")
    print(clf.best_params_)
    print("Grid scores on development set:")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

# Tuning hyper-parameters for precision
Best parameters set found on development set:
{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
Grid scores on development set:
0.124 (+/-0.071) for {'C': 0.001, 'gamma': 0.01, 'kernel': 'rbf'}
0.133 (+/-0.092) for {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
0.133 (+/-0.092) for {'C': 0.001, 'gamma': 0.0001, 'kernel': 'rbf'}
0.133 (+/-0.092) for {'C': 0.001, 'gamma': 1e-05, 'kernel': 'rbf'}
0.124 (+/-0.071) for {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
0.133 (+/-0.092) for {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
0.133 (+/-0.092) for {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.133 (+/-0.092) for {'C': 0.1, 'gamma': 1e-05, 'kernel': 'rbf'}
0.124 (+/-0.071) for {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
0.133 (+/-0.092) for {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
0.133 (+/-0.092) for {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.133 (+/-0.092) for {'C': 0.1, 'gamma': 1e-05, 'kernel': 'rbf'}
0.144 (+/-0.105) for {'C': 10, 'gamma': 0.01, 'ker

In [338]:
clf.best_params_

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [339]:
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X,Y)

all_scores = []

model = SVC(C = 1000, kernel='sigmoid',gamma= 0.01, decision_function_shape='ovo')


for train_index, test_index in skf.split(X,Y):
    model.fit(X[train_index],Y[train_index])
    val_mae = model.score(X[test_index],Y[test_index])
    all_scores.append(val_mae)
    
print(all_scores)



[0.16666666666666666, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666, 0.0, 0.16666666666666666, 0.16666666666666666, 0.3333333333333333, 0.0, 0.3333333333333333]


In [340]:
np.mean(all_scores)

0.19999999999999998

In [341]:
a = eeg_data.iloc[180:240,0:-1]
b = eeg_data.iloc[0:60,0:-1]
X =np.concatenate((a,b),axis=0)

In [342]:
X.shape

(120, 160)

# Decision Trees

In [343]:
X = eeg_data.iloc[:,0:-1]
X1=np.array(X)
Y = eeg_data['target']
Y1= np.array(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X1, Y1, random_state=42)
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(max_features=160,max_leaf_nodes=12,random_state=42)
dt_clf.fit(X_train,Y_train)




DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=160, max_leaf_nodes=12,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [344]:
dt_clf.score(X_train,Y_train)

0.3888888888888889

In [345]:
dt_clf.score(X_test,Y_test)

0.08333333333333333

In [346]:
dt_clf.get_n_leaves()

12

In [347]:
dt_clf.get_depth()

6

# Random forest


In [348]:
from sklearn.ensemble import RandomForestClassifier
X = eeg_data.iloc[:,0:-1]
X1=np.array(X)
Y = eeg_data['target']
Y1= np.array(Y)

X_train, X_test, y_train, y_test = train_test_split(X1, Y1, random_state=42)

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
for name, score in zip(X.columns.tolist(), rnd_clf.feature_importances_):
    print(name, score)

ACh1 0.007222181018495589
ACh2 0.0075504941863516
ACh3 0.0056734950854697005
ACh4 0.005902357339498688
ACh5 0.006972698595767034
ACh6 0.006307475984893574
ACh7 0.005625136488652275
ACh8 0.009460807031296975
ACh9 0.010438309143297924
ACh10 0.009027584841170973
ACh11 0.005279096272489251
ACh12 0.006023286961804617
ACh13 0.007014285053653577
ACh14 0.006429733581715172
ACh15 0.007350816063427298
ACh16 0.006824133025505858
ACh17 0.0054368736599257625
ACh18 0.0
ACh19 0.007255213541507466
ACh20 0.0073632634151842916
ACh21 0.0059737512880667
ACh22 0.007829476991529926
ACh23 0.006372768181636911
ACh24 0.006449131171576413
ACh25 0.006004042163061464
ACh26 0.006194404803220289
ACh27 0.006693397425494127
ACh28 0.004982948797381445
ACh29 0.005108488247931214
ACh30 0.005329084760713388
ACh31 0.008070665588173653
ACh32 0.006123644901994017
BCh1 0.00810389574421663
BCh2 0.012317254698862995
BCh3 0.005308756088457181
BCh4 0.0060523094031496895
BCh5 0.006577805078329592
BCh6 0.007741197979665396
BCh7 0.

In [349]:
rnd_clf.feature_importances_

array([0.00722218, 0.00755049, 0.0056735 , 0.00590236, 0.0069727 ,
       0.00630748, 0.00562514, 0.00946081, 0.01043831, 0.00902758,
       0.0052791 , 0.00602329, 0.00701429, 0.00642973, 0.00735082,
       0.00682413, 0.00543687, 0.        , 0.00725521, 0.00736326,
       0.00597375, 0.00782948, 0.00637277, 0.00644913, 0.00600404,
       0.0061944 , 0.0066934 , 0.00498295, 0.00510849, 0.00532908,
       0.00807067, 0.00612364, 0.0081039 , 0.01231725, 0.00530876,
       0.00605231, 0.00657781, 0.0077412 , 0.00498617, 0.00507939,
       0.00569186, 0.00504344, 0.00580687, 0.00605164, 0.00567488,
       0.00637295, 0.00506765, 0.00499744, 0.0046562 , 0.        ,
       0.0042277 , 0.00559096, 0.00789745, 0.00538847, 0.00575833,
       0.00538834, 0.00652555, 0.00879459, 0.01011772, 0.00576681,
       0.00473728, 0.00423339, 0.00513902, 0.00558956, 0.00675755,
       0.00710655, 0.00750793, 0.00724584, 0.00657301, 0.00604903,
       0.00530757, 0.00645796, 0.00497543, 0.00562959, 0.00769

In [350]:
sum(rnd_clf.feature_importances_)

1.0

In [351]:
rnd_clf.score(X_train,y_train)

1.0

In [352]:
rnd_clf.score(X_test,y_test)

0.03333333333333333

# 3 task Random Forest

In [353]:
X = eeg_data.iloc[180:240,0:-1]
minimum = X.min(axis=0)
X -= minimum
std = X.max(axis=0) - X.min(axis =0)
X /= std
X1 = pd.DataFrame(X)
X1 = X1.fillna(0.0)
X1=np.array(X1)

In [354]:

Y = eeg_data['target'][180:240]
Y1= np.array(Y)


# Feature importances

In [355]:
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, random_state=42)

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
for name, score in zip(X.columns.tolist(), rnd_clf.feature_importances_):
    print(name, score)

ACh1 0.00442915260936767
ACh2 0.007003740467366093
ACh3 0.005347980666967839
ACh4 0.007760624567882081
ACh5 0.007174052065463152
ACh6 0.00470379621345233
ACh7 0.00645401241625775
ACh8 0.005009902271643917
ACh9 0.00636090234104818
ACh10 0.003991904956377621
ACh11 0.007561413470462231
ACh12 0.005982061572261565
ACh13 0.005463319586868718
ACh14 0.0049000579049520545
ACh15 0.006424401300206308
ACh16 0.007942841425726264
ACh17 0.007989405176640289
ACh18 0.0
ACh19 0.005528652289205882
ACh20 0.007368913460456696
ACh21 0.006571116000980828
ACh22 0.007989320134455233
ACh23 0.00852629074301941
ACh24 0.007173991326487878
ACh25 0.0045824697778375605
ACh26 0.005738839572832213
ACh27 0.0061596784217077295
ACh28 0.00415519948446149
ACh29 0.005969379201939564
ACh30 0.008791376192438716
ACh31 0.0032069309114422075
ACh32 0.005817081982370535
BCh1 0.007292408380757848
BCh2 0.007738737111370294
BCh3 0.006886338338822559
BCh4 0.008628358522878394
BCh5 0.007604045078075074
BCh6 0.010352679320623968
BCh7 0.0

In [356]:
rnd_clf.score(X_train,y_train)

1.0

In [357]:
rnd_clf.score(X_test,y_test)

0.0

In [358]:
## decision tree 하나의 예측측
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

0.2


In [359]:
tree_clf.score(X_train,y_train)

1.0

# Voting

In [360]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf = LogisticRegression(solver='liblinear', random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma='auto', probability=True, random_state=42)


##soft voting으로 바꿔줌줌
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gin...
                                        

In [361]:
voting_clf.score(X_train, y_train)

0.8222222222222222

In [362]:
voting_clf.score(X_test, y_test)

0.0

In [363]:

from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.13333333333333333
RandomForestClassifier 0.0
SVC 0.2
VotingClassifier 0.0


# kPCA 차원 축소를 적용한 후 Random Forest 적용해보기 

In [364]:
X = eeg_data.iloc[:,0:-1]
X=np.array(X)

In [365]:
Y = eeg_data["target"]
Y = np.array(Y)

In [366]:
X.shape

(240, 160)

In [469]:

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=None)
lda.fit(X,Y)

X_reduced_lda = lda.transform(X)



In [470]:
X_reduced_lda.shape

(240, 11)

In [471]:
a = pd.DataFrame(X_reduced_lda)

In [472]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced_lda, Y, random_state=42)

rnd_clf = RandomForestClassifier(n_estimators=500)
rnd_clf.fit(X_train, y_train)
for name, score in zip(a.columns.tolist(), rnd_clf.feature_importances_):
    print(name, score)

0 0.12528309235264726
1 0.10498255066946934
2 0.12236409053971473
3 0.09476925685309437
4 0.11626803999065415
5 0.0986906218862369
6 0.0841639608679486
7 0.06763212330027246
8 0.06942757076745575
9 0.06953519729283274
10 0.04688349547967372


In [473]:
rnd_clf.score(X_train,y_train)

1.0

In [474]:
rnd_clf.score(X_test,y_test)

0.9166666666666666

In [475]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf = LogisticRegression(solver='liblinear', random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma='auto', probability=True, random_state=42)


##soft voting으로 바꿔줌줌
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gin...
                                        

In [476]:
voting_clf.score(X_train, y_train)

0.9944444444444445

In [477]:
voting_clf.score(X_test, y_test)

0.8833333333333333

In [478]:

from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.85
RandomForestClassifier 0.8666666666666667
SVC 0.8333333333333334
VotingClassifier 0.8833333333333333
